# Dependencies

In [3]:
from fsutils import run_sf_sql as rp, config, email, adls_gen2, log
import pandas as pd
from datetime import datetime
from snowflake.connector.pandas_tools import write_pandas

# Get the SF connection and cursor

Inside config.py slfsvc-twa07
printing snowflake connection dic {'tenant': 'fabb61b8-3afe-4e75-b934-a47f782b8cd7', 'client_id': '5617df5a-e4aa-4898-8260-9ac51b67e1c0', 'akssecret': 'prod-fspe-sp-secret', 'vault': 'kv-rtlfspekv0701slfsvc07', 'sfsecretkey': 'sfkey', 'sfkeypass': 'sfpasskey'}
Initiating azcopy login
az login succeeded


In [4]:
conn, _=rp.get_connection("notebook-xlarge")

Inside config.py slfsvc-twa07
printing snowflake connection dic {'SF_USER_NAME': 'sa_slfsvc_fsca_app_twa_07', 'SF_ACCOUNT': 'cvsfrontstore.east-us-2.privatelink', 'SF_WAREHOUSE': 'WH_FSCA_TWA07_XL_BATCH', 'SF_USER_ROLE': 'ROLE_FSCA_TWA07_BATCH_FR', 'SF_DB': 'CORE_FS', 'SF_AUDIT_SCHEMA': 'WORK_FS'}
printing sfparms {'SF_USER_NAME': 'sa_slfsvc_fsca_app_twa_07', 'SF_ACCOUNT': 'cvsfrontstore.east-us-2.privatelink', 'SF_WAREHOUSE': 'WH_FSCA_TWA07_XL_BATCH', 'SF_USER_ROLE': 'ROLE_FSCA_TWA07_BATCH_FR', 'SF_DB': 'CORE_FS', 'SF_AUDIT_SCHEMA': 'WORK_FS'}
2025-04-15 14:01:31,443 - fsutils.run_sf_sql - INFO - Establishing Snowflake Connection


INFO:fsutils.run_sf_sql:Establishing Snowflake Connection


Getting snowflake key file from Azure Key Vault
Getting snowflake key pass from Azure Key Vault


ERROR:snowflake.connector.ocsp_snowflake:Failed to get OCSP response after 1 attempt. Consider checking for OCSP URLs being blocked
 {'driver': 'PythonConnector', 'version': '3.12.4', 'eventType': 'RevocationCheckFailure', 'eventSubType': 'OCSPResponseFailedToConnectCacheServer|OCSPResponseFetchFailure', 'sfcPeerHost': 'cvsfrontstore.east-us-2.privatelink.snowflakecomputing.com', 'certId': 'MEcwBwYFKw4DAhoEFKfEuLPcW7VYHqfX8TrFafVvSNeJBBR0hYDAZsffN97PvSk3qgMdvu3NFwIQASaIWIfhcMLpgMqT/CPqlQ==', 'ocspRequestBase64': 'ME8wTTBLMEkwRzAHBgUrDgMCGgQUp8S4s9xbtVgep9fxOsVp9W9I14kEFHSFgMBmx9833s+9KTeqAx2+7c0XAhABJohYh+FwwumAypP8I+qV', 'ocspResponderURL': 'http://ocsp.digicert.com', 'errorMessage': '254014: Failed to get OCSP response after 1 attempt.', 'insecureMode': False, 'failOpen': True, 'cacheEnabled': True, 'cacheHit': False}
ERROR:snowflake.connector.ocsp_snowflake:Failed to get OCSP response after 1 attempt. Consider checking for OCSP URLs being blocked
 {'driver': 'PythonConnector', 'vers

2025-04-15 14:01:54,871 - fsutils.run_sf_sql - INFO -  Getting Snowflake Cursor


INFO:fsutils.run_sf_sql: Getting Snowflake Cursor


2025-04-15 14:01:54,872 - fsutils.run_sf_sql - INFO - Got the cursor


INFO:fsutils.run_sf_sql:Got the cursor


In [5]:
!pip install openpyxl

Looking in indexes: https://nexus-az.cvshealth.com:9443/repository/pypi-proxy/simple/, https://nexus-az.cvshealth.com:9443/repository/Fsutils/simple, https://nexus-az.cvshealth.com:9443/repository/Niagara/simple
  Using cached https://nexus-az.cvshealth.com:9443/repository/pypi-proxy/packages/openpyxl/3.1.5/openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached https://nexus-az.cvshealth.com:9443/repository/pypi-proxy/packages/et-xmlfile/2.0.0/et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


# Write new items results to snowflake

In [6]:
# # Get the current date and time in ddmmyyy_hhmm format
# now = datetime.now().strftime('%Y%m%d_%H%M')
# filename_suffix = f"_{now}"  # This will be appended to the filename

# # Read Excel file
# ns_data_new_items = pd.read_excel("final_need_states_new_items-20250407161801.xlsx")

# # Convert column names to uppercase
# ns_data_new_items.columns = ns_data_new_items.columns.str.upper()

# # Ensure all ATTRIBUTE columns are strings
# for attribute in ['ATTRIBUTE_1', 'ATTRIBUTE_2', 'ATTRIBUTE_3', 'ATTRIBUTE_4', 'ATTRIBUTE_5', 'ATTRIBUTE_6']:
#     ns_data_new_items[attribute] = ns_data_new_items[attribute].astype(str)

In [7]:
# ns_data_new_items.to_csv("temp.csv",index=False)

In [8]:
# # Write to SQL database with updated filename
# write_pandas(conn, ns_data_new_items, "NEED_STATES_NEW_ITEMS" + filename_suffix, 
#              database="DL_FSCA_SLFSRV", schema="TWA07", auto_create_table=True, overwrite=True)

In [9]:
# print("NEED_STATES_NEW_ITEMS" + filename_suffix)

# Need States Input for Clustering

### *Filtering Matt*

In [10]:
import pandas as pd
import logging
import string

# ------------------------------------------------------------------------------
# Set up logging
# ------------------------------------------------------------------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ------------------------------------------------------------------------------
# Configuration: alternate names for product_id
# ------------------------------------------------------------------------------
ALTERNATE_PRODUCT_ID_COLUMNS = ["item_nbr", "item_no_nbr", "item_id"]

# ------------------------------------------------------------------------------
# Helper Function to Standardize "to_be_dropped"
# ------------------------------------------------------------------------------
def standardize_to_be_dropped(value):
    """
    Remove punctuation, convert to lowercase, and trim whitespace.
    """
    val_str = str(value)
    # Remove punctuation
    val_str_no_punc = val_str.translate(str.maketrans("", "", string.punctuation))
    # Lowercase and strip
    return val_str_no_punc.lower().strip()

# ------------------------------------------------------------------------------
# Main Processing Function
# ------------------------------------------------------------------------------
def process_laundry_files(file_paths):
    """
    Processes each Excel file in file_paths:
      - Reads the file.
      - Reconciles the product_id column (renames alternate columns if needed).
      - Standardizes the 'to_be_dropped' column and logs any unexpected values.
      - Filters rows where 'to_be_dropped' (standardized) equals 'no'.
      - Adds a 'data_source' column with the file name.

    Returns two DataFrames:
      1. full_df: The union of all columns from the files (with data_source).
      2. minimal_df: Only 'product_id' and 'data_source'.
    """
    full_frames = []
    minimal_frames = []

    for file_path in file_paths:
        try:
            df = pd.read_excel(file_path)
            logger.info(f"Successfully read file: {file_path}")
        except Exception as e:
            logger.error(f"Error reading file {file_path}: {e}")
            continue

        # --- Reconcile product_id ---
        if "product_id" not in df.columns:
            found_alternate = False
            for alt in ALTERNATE_PRODUCT_ID_COLUMNS:
                if alt in df.columns:
                    logger.warning(
                        f"File {file_path}: No 'product_id' column found. Renaming '{alt}' to 'product_id'."
                    )
                    df.rename(columns={alt: "product_id"}, inplace=True)
                    found_alternate = True
                    break
            if not found_alternate:
                logger.warning(f"File {file_path}: No 'product_id' column found. Skipping file.")
                continue

        # --- Standardize and check 'to_be_dropped' ---
        if "to_be_dropped" not in df.columns:
            logger.warning(f"File {file_path}: 'to_be_dropped' column not found. Skipping file.")
            continue

        df["to_be_dropped_std"] = df["to_be_dropped"].apply(standardize_to_be_dropped)
        # Log unexpected values (only 'yes' or 'no' expected)
        unexpected_vals = df.loc[
            ~df["to_be_dropped_std"].isin(["yes", "no"]), "to_be_dropped_std"
        ].unique()
        if len(unexpected_vals) > 0:
            logger.warning(
                f"File {file_path}: Unexpected values in 'to_be_dropped': {list(unexpected_vals)}"
            )

        # --- Filter rows where to_be_dropped is 'no'
        df_filtered = df[df["to_be_dropped_std"] == "no"].copy()
        if df_filtered.empty:
            logger.info(f"File {file_path}: No rows with 'to_be_dropped' == 'no' after filtering.")
            continue

        # --- Add data_source column ---
        df_filtered["data_source"] = file_path

        # --- For full union output, drop the internal standardization column ---
        if "to_be_dropped_std" in df_filtered.columns:
            df_filtered.drop(columns=["to_be_dropped_std"], inplace=True)

        full_frames.append(df_filtered)

        # --- Create minimal output (only product_id and data_source) ---
        minimal_df = df_filtered[["product_id", "data_source"]].copy()
        minimal_frames.append(minimal_df)

    # --- Concatenate all processed DataFrames ---
    if full_frames:
        full_df = pd.concat(full_frames, ignore_index=True, sort=False)
    else:
        full_df = pd.DataFrame()

    if minimal_frames:
        minimal_df = pd.concat(minimal_frames, ignore_index=True)
    else:
        minimal_df = pd.DataFrame(columns=["product_id", "data_source"])

    return full_df, minimal_df

In [11]:
# Define the three Excel file paths
file_paths = [
        "Fabric Enhancers.xlsx",
        "Laundry Bleach.xlsx",
        "Laundry.xlsx",
]

# Process the files
full_output_df, minimal_output_df = process_laundry_files(file_paths)

# Write out the minimal file (only product_id and data_source)
minimal_output_file = "combined_product_ids.csv"
try:
  minimal_output_df.to_csv(minimal_output_file, index=False)
  logger.info(f"Minimal output written to: {minimal_output_file}")
except Exception as e:
  logger.error(f"Error writing {minimal_output_file}: {e}")

# Write out the full union file (all unique columns plus data_source)
full_output_file = "combined_full.csv"
try:
  full_output_df.to_csv(full_output_file, index=False)
  logger.info(f"Full union output written to: {full_output_file}")
except Exception as e:
  logger.error(f"Error writing {full_output_file}: {e}")

# Optionally, print the shapes of the outputs for verification
print("Minimal output shape:", minimal_output_df.shape)
print("Full output shape:", full_output_df.shape)

Minimal output shape: (203, 2)
Full output shape: (203, 15)


In [16]:
minimal_output_df.head()

,product_id,data_source
0,309865,Fabric Enhancers.xlsx
1,493249,Fabric Enhancers.xlsx
2,436367,Fabric Enhancers.xlsx
3,634885,Fabric Enhancers.xlsx
4,369415,Fabric Enhancers.xlsx


In [14]:
full_output_df.head()

,product_id,product_name,product_description,category,subcategory,Brand_standardized,Form_standardized,Load Count_standardized,Machine Compatibility_standardized,Price_standardized,Scent_standardized,New Item,to_be_dropped,out_of_scope,data_source
0,309865,Downy Rinse & Refresh Laundry Odor Remover and...,Product Description: Tired of stubborn odors t...,Household,Laundry Supplies,Downy,Liquid Fabric Softener,31-60,HE,10.01-15,Floral & Botanical,False,No,No,Fabric Enhancers.xlsx
1,493249,Bounce Free & Gentle Fabric Softener Dryer She...,Product Description: Sensitive skin? Bounce Fr...,Household,Laundry Supplies,Other,Dryer Sheets,91-130,Unspecified,5.01-10,Unscented,False,No,No,Fabric Enhancers.xlsx
2,436367,"Gain In-Wash Scent Booster Beads, Happy Hibisc...",Product Description: Sniff the good vibes. Tra...,Household,Laundry Supplies,Gain,Scent Enhancers,0-30,HE,5.01-10,Floral & Botanical,False,No,No,Fabric Enhancers.xlsx
3,634885,"Downy Light Scent In-Wash Scent Booster Beads,...","Product Description: Finally, long-lasting sce...",Household,Laundry Supplies,Downy,Scent Enhancers,0-30,HE,5.01-10,Fresh,False,No,No,Fabric Enhancers.xlsx
4,369415,"Downy Fabric Softener Dryer Sheets, April Fres...",Product Description: Downy April Fresh Dryer S...,Household,Laundry Supplies,Downy,Dryer Sheets,91-130,Unspecified,5.01-10,Floral & Botanical,False,No,No,Fabric Enhancers.xlsx


In [4]:
# ns_data = pd.read_excel("final_need_states_new_items-20250403122120.xlsx")
from fsutils import run_sf_sql as rp, config, email, adls_gen2, log
conn, _=rp.get_connection("notebook-xlarge")
ns_data = pd.read_sql("select * from DL_FSCA_SLFSRV.TWA07.FINAL_NEED_STATES_NEW_ITEMS", conn)

In [5]:
# ns_data = pd.read_excel("final_need_states_new_items-20250407161801.xlsx")

In [24]:
# ns_data.loc[ns_data['CATEGORY'].isin(['Hair Care - Hair Styling', 'Hair Care - Shampoo and Conditioner']), 'CATEGORY'] = 'Hair Care'

In [6]:
ns_data = ns_data.query("`New Item` == False")

ns_data.columns = ns_data.columns.str.upper() #make column names uppercase

ns_data = ns_data[['PRODUCT_ID', 'CATEGORY', 'NEED_STATE', 'CDT', 'ATTRIBUTE_1',
       'ATTRIBUTE_2', 'ATTRIBUTE_3', 'ATTRIBUTE_4', 'ATTRIBUTE_5',
       'ATTRIBUTE_6',]].drop_duplicates()

ns_data = ns_data[~ns_data['NEED_STATE'].isna()]

# Remember to change the date!
ns_data.to_csv("/home/jovyan/fsassortment/store_clustering/data/need_states_mapping_20250414_AM.csv",index=False)

# Output

In [7]:
for attribute in ['ATTRIBUTE_1', 'ATTRIBUTE_2', 'ATTRIBUTE_3', 'ATTRIBUTE_4', 'ATTRIBUTE_5', 'ATTRIBUTE_6']:
    ns_data[attribute] = ns_data[attribute].astype(str)

write_pandas(conn, ns_data, "NEED_STATES_20250414_AM", database="DL_FSCA_SLFSRV", schema="TWA07", auto_create_table=True, overwrite=True)

/tmp/ipykernel_57/53748760.py:4: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(conn, ns_data, "NEED_STATES_20250414_AM", database="DL_FSCA_SLFSRV", schema="TWA07", auto_create_table=True, overwrite=True)


(True,
 1,
 14677,
 [('ximinalaak/file0.txt',
   'LOADED',
   14677,
   14677,
   1,
   0,
   None,
   None,
   None,
   None)])